In [ ]:
import openai

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
import os

In [7]:
response = client.chat.completions.create(
    model ='gpt-3.5-turbo-0125',
    messages = [{"role": "user", "content": "Is it too late to join the course ?"}]
)

In [10]:
response.choices[0].message.content

'It depends on the specific course and its enrollment policies. You may need to check with the course provider or institution to see if they are still accepting students. Some courses may have specific enrollment periods while others may allow for late enrollment.'